# 0.0 - Logistic Regression

Os parâmetros utilizados para treinar o algoritmo **Logistic Regression** serão o *inverso da força de regularização* `C`, o `solver` (algoritmo utilizado no problema de otimização) e o `max_iter` (número máximo de iterações escolhido para a convergência do algoritmo de otimização).</br>Os valores padrão destes parâmetros são `C=1.0`, `solver=lbfgs` e `max_iter=100`. Para mais detalhes, veja a documentação oficial em:</br></br>https://scikit-learn.org/1.5/modules/generated/sklearn.linear_model.LogisticRegression.html</br></br>
Para os cálculos das métricas de performance, considerei a classe positiva como sendo a **0**.</br>Por padrão, o *sklearn* toma como sendo a classe **1**.

# 1.0 - Importando bibliotecas

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import metrics as mt
from itertools import product

from matplotlib import pyplot as pl

# 2.0 - Carregando os dados

In [2]:
# Treino
X_train = pd.read_csv('train/X_training.csv')
y_train = pd.read_csv('train/y_training.csv')

# Validação
X_val = pd.read_csv('validation/X_validation.csv')
y_val = pd.read_csv('validation/y_validation.csv')

# Teste
X_test = pd.read_csv('test/X_test.csv')
y_test = pd.read_csv('test/y_test.csv')

# 3.0 - Funções

In [3]:
def metrics(y, y_pred, step, algorithm):
    """
    Calcula várias métricas de avaliação de um modelo de classificação e as retorna em um DataFrame.

    Parâmetros:
    -----------
    y : array-like
        Valores da variável resposta.
    
    y_pred : array-like
        Valores preditos pelo modelo.
    
    step : str
        Etapa do processo de modelagem (ex: 'treino', 'validação', 'teste').
    
    algorithm : str
        Nome do algoritmo usado.

    Retorno:
    --------
    df_metrics : pandas.DataFrame
        DataFrame contendo as métricas calculadas:
        - 'Algorithm': Nome do algoritmo.
        - 'Step': Etapa do processo (treino, validação ou teste).
        - 'Accuracy': Valor da Acurácia.
        - 'Precision': Valor da Precision.
        - 'Recall': Valor da Recall.
        - 'F1-Score': Valor de F1-Score.
    """
    # Calcula as métricas
    acuracia  = mt.accuracy_score( y, y_pred )
    precision = mt.precision_score( y, y_pred, pos_label = 0 )
    recall    = mt.recall_score( y, y_pred, pos_label = 0 )
    f1        = mt.f1_score( y, y_pred, pos_label = 0 )

    # Cria o DataFrame com as métricas
    df_metrics = pd.DataFrame({   
        'Algorithm': [algorithm],
        'Step': [step],
        'Accuracy': [np.round(acuracia, 4)],
        'Precision': [np.round(precision, 4)],
        'Recall': [np.round(recall, 4)],
        'F1-Score': [np.round(f1, 4)]
    })
    
    return df_metrics

def exporta_excel(nome_algoritmo):
    """
    Exporta os DataFrames de métricas para arquivos Excel em diferentes pastas.

    Args:
        nome_algoritmo (str): Nome do algoritmo utilizado.

    Returns:
        None
    """
    etapas = ['train', 'validation', 'test']
    df_list = [df_metrics_train, df_metrics_val, df_metrics_test]

    for etapa, df in zip(etapas, df_list):
        df.to_excel(f'{etapa}/metrics/metrics_{etapa}_{nome_algoritmo}.xlsx', index=False)

# 4.0 - Ajustando os dados

In [4]:
# Como o atributo 'id' não contribui para o aprendizado do modelo, iremos retirá-lo.
X_train, X_val, X_test = [ df.drop(columns = ['id']) for df in [X_train, X_val, X_test] ]

# renomeia a coluna '0' da variável resposta com o nome original 'satisfaction'
y_train, y_val, y_test = [ df.rename(columns = {'0': 'satisfaction'}) for df in [y_train, y_val, y_test] ]

# transformação do formato DataFrame para Series, essencialmente um array unidimensional
y_train, y_val, y_test = [ df.loc[:, 'satisfaction'] for df in [y_train, y_val, y_test] ]

# 5.0 - Desempenho do modelo

## 5.1 - Dados de treino

In [5]:
# Definição do modelo
logistic_regression_train = LogisticRegression( random_state = 0 )

# treinamento do algoritmo. 
logistic_regression_train.fit(X_train, y_train)

# Predição sobre os dados de treino
y_pred_train = logistic_regression_train.predict(X_train)

# Métricas
df_metrics_train = metrics(y_train, y_pred_train, 'Train', 'Logistic Regression')
df_metrics_train

,Algorithm,Step,Accuracy,Precision,Recall,F1-Score
0,Logistic Regression,Train,0.8753,0.8785,0.905,0.8916


## 5.2 - Dados de validação

In [6]:
# Definição do modelo
logistic_regression_val = LogisticRegression( random_state = 0 )

# treinamento do algoritmo. 
logistic_regression_val.fit(X_train, y_train)

# Predição sobre os dados de validação
y_pred_val = logistic_regression_val.predict(X_val)

# Métricas
df_metrics_val = metrics(y_val, y_pred_val, 'Validation', 'Logistic Regression')
df_metrics_val

,Algorithm,Step,Accuracy,Precision,Recall,F1-Score
0,Logistic Regression,Validation,0.8742,0.8777,0.9039,0.8906


## 5.3 - Ajuste fino dos hiperparâmetros (Etapa de *Fine Tuning*)

In [7]:
# lista para os valores de 'C' 
C_list = [0.20, 0.50, 0.85, 1.00, 1.50]

# lista com os solvers a serem testados 
solver_list = ['lbfgs', 'newton-cg', 'newton-cholesky']

# lista com os valores de 'max_iter'
max_iter_list = [100, 500]

# Criar DataFrame para armazenar resultados
metrics_df = pd.DataFrame(columns=['C', 'solver', 'max_iter', 'accuracy', 'precision', 'recall', 'f1_score'])

# Usar 'itertools.product' para combinar os hiper-parâmetros
for m, n, o in product(C_list, solver_list, max_iter_list):
    # Definir e treinar o algoritmo Logistic Regression
    model = LogisticRegression(C=m, solver=n, max_iter=o, random_state=0)
    model.fit(X_train, y_train)

    # Predição sobre os dados de validação
    y_pred = model.predict(X_val)

    # Calcular métricas de performance
    acuracia  = mt.accuracy_score(y_val, y_pred)
    precision = mt.precision_score(y_val, y_pred, pos_label=0)
    recall    = mt.recall_score(y_val, y_pred, pos_label=0)
    f1        = mt.f1_score(y_val, y_pred, pos_label=0)

    # Armazenar os resultados em uma nova linha como DataFrame
    new_row = pd.DataFrame({
        'C': [m],
        'solver': [n],
        'max_iter': [o],
        'accuracy': [acuracia],
        'precision': [precision],
        'recall': [recall],
        'f1_score': [f1]
    })

    # Concatenar a nova linha ao DataFrame existente
    metrics_df = pd.concat([metrics_df, new_row], ignore_index=True)

In [8]:
# DataFrame com os valores das métricas de performance para os hiper-parâmetros testados
metrics_df

,C,solver,max_iter,accuracy,precision,recall,f1_score
0,0.20,lbfgs,100,0.873999,0.877495,0.903805,0.890455
1,0.20,lbfgs,500,0.873999,0.877495,0.903805,0.890455
2,0.20,newton-cg,100,0.873773,0.877364,0.903521,0.890250
3,0.20,newton-cg,500,0.873773,0.877364,0.903521,0.890250
4,0.20,newton-cholesky,100,0.873838,0.877419,0.903578,0.890306
5,0.20,newton-cholesky,500,0.873838,0.877419,0.903578,0.890306
6,0.50,lbfgs,100,0.873838,0.877378,0.903634,0.890312
7,0.50,lbfgs,500,0.873838,0.877378,0.903634,0.890312
8,0.50,newton-cg,100,0.873773,0.877447,0.903407,0.890238
9,0.50,newton-cg,500,0.873773,0.877447,0.903407,0.890238


Como o objetivo principal da pesquisa é identificar os clientes insatisfeitos para tomar ações corretivas, o **recall** é a figura de mérito mais relevante.</br>Assim, um alto **recall** garante que a maioria dos clientes insatisfeitos seja corretamente identificada.</br>
Os melhores valores de **C**, **solver** e **max_iter** são aqueles que produzem o maior valor da métrica de interesse.

In [10]:
metrics_df.sort_values(by='recall', ascending=False).head(5)

,C,solver,max_iter,accuracy,precision,recall,f1_score
19,1.0,lbfgs,500,0.874159,0.877695,0.903861,0.890586
18,1.0,lbfgs,100,0.874159,0.877695,0.903861,0.890586
0,0.2,lbfgs,100,0.873999,0.877495,0.903805,0.890455
25,1.5,lbfgs,500,0.874159,0.877737,0.903805,0.890580
24,1.5,lbfgs,100,0.874159,0.877737,0.903805,0.890580


Observando as duas linhas inciais do dataset acima, perceba que os parâmetros `C = 1.0`, `solver = lbfgs`, `max_iter = 100` ou `500` retornam o melhor valor de **recall**, *0.903861*. Nesse caso, tanto faz utilizar 100 ou 500 para `max_iter`. Optamos pelo primeiro caso. 

In [11]:
# Melhores hiperparametros: 'C' = 1.0, 'solver' = 'lbfgs' e 'max_iter' = 100
best_c = 1.0
best_solver = 'lbfgs'
best_max_iter = 100

## 5.4 - Desempenho do modelo para os dados de teste

In [14]:
# Definição do modelo
logistic_regression_test = LogisticRegression( 
                                 C = best_c, 
                                 solver = best_solver,
                                 max_iter = best_max_iter,
                                 random_state=0
)

# Juntar os dados de treino e validação
logistic_regression_test.fit( pd.concat( [X_train, X_val] ),
                              pd.concat( [y_train, y_val] ) )

# Predições sobre os dados de teste
y_pred_test = logistic_regression_test.predict( X_test )

# Métricas
df_metrics_test = metrics(y_test, y_pred_test, 'Test', 'Logistic Regression')
df_metrics_test

,Algorithm,Step,Accuracy,Precision,Recall,F1-Score
0,Logistic Regression,Test,0.8711,0.873,0.9014,0.887


# 6.0 - Gera as planilhas com os resultados

In [15]:
exporta_excel('logistic_regression')